In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

In [2]:
file_path = 'D:\healthcare_project\Healthcare1.pdf'

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
C:\Users\vrajt\AppData\Local\Temp\ipykernel_10376\3972802260.py:1: SyntaxWarning: invalid escape sequence '\h'
  file_path = 'D:\healthcare_project\Healthcare1.pdf'


In [3]:
import os


In [4]:
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")
     

In [5]:

if not file_path.endswith(".pdf"):
    raise ValueError(f"Invalid file type: {file_path}. Expected a PDF file.")

In [6]:
loader = PyPDFLoader(file_path)


In [7]:
data = loader.load()


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)


In [9]:
text = text_splitter.split_documents(data)


In [10]:
embeddings = FastEmbedEmbeddings(model_name='BAAI/bge-small-en-v1.5')


C:\Users\vrajt\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]C:\Users\vrajt\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vrajt\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-small-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either ne

In [11]:
db = FAISS.from_documents(text, embeddings)


In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [13]:
from groq import Groq



In [14]:
with open('.env', 'w') as f:
    f.write('GROQ_API_KEY=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')

In [15]:
from dotenv import load_dotenv


In [16]:
load_dotenv()


True

In [17]:

from langchain_groq import ChatGroq
     


In [18]:
client = Groq(api_key=os.getenv('GROQ_API_KEY'))


In [19]:
llm = ChatGroq(model_name='llama3-70b-8192')


In [20]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\vrajt\AppData\Local\Temp\ipykernel_10376\1046491335.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [21]:
prompt_template = (
    "You are a helpful assistant. Greet the user and answer the following questions.\n"
    "Chat history:\n{chat_history}\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n"
)
     

In [22]:
prompt = PromptTemplate(template=prompt_template, input_variables=['chat_history', 'context', 'question'])


In [23]:

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    combine_docs_chain_kwargs={'prompt': prompt},
    retriever=retriever
)

In [24]:
import gradio as gr


In [25]:

def chat_with_bot(user_input, history=[]):
  history = history or []
  result = qa(user_input)
  bot_response = result['answer']
  history.append((user_input, bot_response))
  return history, history
     

In [26]:

with gr.Blocks() as chatbot_ui:
    gr.Markdown("#Chat with Your Healthcare RAG Bot")
    chat_history = gr.Chatbot(label="Chatbot")
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...")
    submit_button = gr.Button("Send")
    submit_button.click(chat_with_bot, inputs=[user_input, chat_history], outputs=[chat_history, chat_history])

C:\Users\vrajt\AppData\Roaming\Python\Python312\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [28]:
chatbot_ui.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
